# Visualizando e operando com nuvens de pontos em Caderno Jupyter

In [1]:
# Preparar o ambiente no Google Drive:
#from google.colab import drive
#drive.mount('/content/gdrive')

Instalar as seguintes bibliotecas se for rodar no Colab:

In [2]:
# Instalar bibliotecas que iremos utilizar
# Bibliotecas para uso geral
#!pip install numpy
#!pip install pandas
# Biblioteca para manipulação e vizualização de nuvens de pontos
#!pip install ipyvolume 
#!pip install pyntcloud
#!pip install open3d
#!ls # para importar dados do Google Drive

In [1]:
# Importar Bibliotecas
import numpy as np
import pandas as pd
import ipyvolume as ipv
import open3d as o3d
from open3d import JVisualizer
from pyntcloud import PyntCloud as pytc
import copy
import time

# Como importar a nuvem?

Uma nuvem de pontos pode se apresentar em diversos formatos, o mais básico desses formatos é simplesmente um arquivo ascii (txt.) com 3 colunas contendo xyz. Quatro das bibliotecas utilizadas permitem esse tipo de importação, mas algumas são mais eficientes que outras. Outro problema diz respeito à formatação e tipo de dado disponível, que pode ser de simples precisão (float32) ou dupla precisão (flaot64). Vamos aos testes: importar uma nuvem pequena (269 mil pontos, colunas separadas por vírgulas) com todas as bibliotecas que permitem isso:

# Import pela Numpy:

A numpy lê a nuvem como uma matriz de 269 mil linhas e 3 colunas, é a mais lenta das formas de importação, demora cerca de 2,1 segundos para ler uma nuvem com 268879 pts.

In [2]:
tempo_i = time.time()
pc_numpy  = np.loadtxt('/home/rubens/rubens_virtual_enviroment/nuvens-LiDAR/nuvem_airborne-LiDAR_no_centroide.xyz', dtype='float', delimiter=',')
tempo_f = time.time() - tempo_i
print(pc_numpy)
print(len(pc_numpy))
print("Tempo para ler a nuvem: %.3f" %tempo_f)

[[ -53.03132588  185.44361056    0.65855475]
 [ -53.25012588  185.56861056    0.64455475]
 [ -53.45322588  185.63111056    0.65455475]
 ...
 [  51.87487412 -181.99388944   -2.18944525]
 [  51.59367412 -181.93138944   -2.22444525]
 [  51.29677412 -181.80638944   -2.20144525]]
268879
Tempo para ler a nuvem: 2.214


# Import pela Pandas:

Um data frame precisa de índices para nomear as colunas dos dados, então vamos nomear as colunas com os nomes x, y e z para que a primeira linha de coordenadas não seja interpretada como índices de colunas. A leitura é 10x mais rápida do que pela numpy.

In [3]:
tempo_i = time.time()
pc_pandas = pd.read_csv('/home/rubens/rubens_virtual_enviroment/nuvens-LiDAR/nuvem_airborne-LiDAR_no_centroide.xyz', names=["x","y","z"])
tempo_f = time.time() - tempo_i
print(pc_pandas)
type(pc_pandas)
print("Tempo para ler a nuvem: %.3f" %tempo_f)

                x           y         z
0      -53.031326  185.443611  0.658555
1      -53.250126  185.568611  0.644555
2      -53.453226  185.631111  0.654555
3      -53.672026  185.693611  0.650555
4      -53.859426  185.756111  0.638555
...           ...         ...       ...
268874  52.421774 -182.181389 -2.223445
268875  52.156174 -182.118889 -2.237445
268876  51.874874 -181.993889 -2.189445
268877  51.593674 -181.931389 -2.224445
268878  51.296774 -181.806389 -2.201445

[268879 rows x 3 columns]
Tempo para ler a nuvem: 0.189


# Import pela Pyntcloud:

Por trás da Pyntcloud roda a função read_csv da pandas para importar dados, mas a pyntcloud cria um objeto próprio. 
Precisamos que o dado esteja formatado com colunas que possuam nomes x, y e z. Isso é feito dando o argumento 'names'  à função de leitura da PyntCloud (que igual à read_csv da pandas). O tempo de leitura é semelhante também.

In [4]:
tempo_i = time.time()
pc_pyntcloud = pytc.from_file('/home/rubens/rubens_virtual_enviroment/nuvens-LiDAR/nuvem_airborne-LiDAR_no_centroide.xyz',names=["x","y","z"])
tempo_f = time.time() - tempo_i
print(pc_pyntcloud)
type(pc_pyntcloud)
print("Tempo para ler a nuvem: %.3f" %tempo_f)

PyntCloud
268879 points with 0 scalar fields
0 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: 3.0625871847689797e-09, -4.6036352185236944e-11, 3.3030529579523296e-14
Other attributes:

Tempo para ler a nuvem: 0.198


# Import pela Open3D:

A Open3D consegue ler arquivos pcd, ply e ascii como: xyz, xyzrgb, xyzn e pts. Todavia, no caso dos tipos ascii, ela não consegue ler dados separados por vírgula. Podemos contornar isso utilizando a função da Open3d que lê a matriz numpy gerada anteriormente:

In [5]:
tempo_i = time.time()
pc_open3d = o3d.geometry.PointCloud()
pc_open3d.points = o3d.utility.Vector3dVector(pc_numpy)
tempo_f = time.time() - tempo_i
print(pc_open3d)
type(pc_open3d)
print("Tempo para ler a nuvem: %.3f" %tempo_f)

geometry::PointCloud with 268879 points.
Tempo para ler a nuvem: 0.008


Ou, simplemente lendo a nuvem no formato pcd (é pelo menos 3x mais rápida que a Pandas):

In [6]:
tempo_i = time.time()
pc_open3d = o3d.io.read_point_cloud("/home/rubens/rubens_virtual_enviroment/nuvens-LiDAR/nuvem_airborne-LiDAR_no_centroide.pcd")
tempo_f = time.time() - tempo_i
print(pc_open3d)
type(pc_open3d)
print("Tempo para ler a nuvem: %.3f" %tempo_f)

geometry::PointCloud with 268879 points.
Tempo para ler a nuvem: 0.068


# Visualização com a PyntCloud: 

A visualização da pyntcloud aparece com os pontos em tamanhos exagerados, mas é possível alterar isso facilmente e por algum valor entre 0 e 1 para que a nuvem apareça normalmente. Não funciona no colab, ele desconectará assim que rodarmos o seguinte comando:

In [7]:
pc_pyntcloud.plot()

/home/rubens/.local/lib/python3.6/site-packages/pythreejs/traits.py:191: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(3.0625871847689797e-09, 185.75611055521347, …

# Visualização com a ipyVolume: 

A ipyVolume permite animações e muitos outros recursos da ipywidgets incorporados na sua implementação.
Todavia, a renderização é mais complexa e pode ficar lenta dependendo do seu hardware. Para plotar os pontos precisamos das colunas da matriz numpy:

In [8]:
x = pc_numpy[:,0];y = pc_numpy[:,1];z = pc_numpy[:,2]
ipv.quickscatter(x, y, z, size=0.5, marker="sphere")

# Visualização com a Open3d: 

A Open3d não suporta visualizações dentro de cadernos jupyter, mas mesmo assim desenha a nuvem em uma janela a parte. Essa nuvem não aparece se os dados estiverem em coordenadas UTM.

In [9]:
o3d.visualization.draw_geometries([pc_open3d])

# Kd-trees & Octrees: Pytncloud

Kd-trees e octrees são estruturas muito utilizadas quando manipulamos nuvens de pontos. A maioria dos filtros e diversas outras funcionalidades necessitam de kd-trees e octrees para funcionar. O nome tree (árvore) se deve à forma como o objeto é criado através de subdivisões do espaço. Apenas as bibliotecas Open3D e Pyncloud, específicas para nuvens de pontos, trabalham com este objeto.

No exemplo abaixo utilizamos uma kd-tree para selecionar os k vizinhos mais próximos de cada ponto da nuvem. A estrutura gerada é uma matriz com k colunas com os índices dos pts mais próximos.

In [12]:
# Adicionar estrutura kdtree na nuvem:
kdtree_pyntcloud = pc_pyntcloud.add_structure("kdtree")
# Guardar k vizinhos de cada ponto:
k_vizinhos = pc_pyntcloud.get_neighbors(k=6,kdtree=kdtree_pyntcloud)
print(k_vizinhos)

[[     8      1      7      2      6      3]
 [     7      8      2      0      6      3]
 [     6      7      1      3      5      8]
 ...
 [268847 268848 268877 268875 268846 268849]
 [268846 268847 268876 268878 268845 268848]
 [268845 268846 268877 268844 268847 268876]]


A função abaixo permite subdividir o espaço ocupado pela nuvem com uma octree. Os parâmetros da subdivisão podem ser a quantidade de voxels finais desejados em cada eixo (n_x=1, n_y=1, n_z=1) ou o tamanho dos voxels criados em cada eixo (size_x=1.0, size_y=1.0, size_z=1.0). Colocar valores pequenos nos tamanhos (abaixo de 0.5) ou muito grandes nas quantidades finais (acima de 100) costuma travar o Jupyter por falta de memória. Setar 'None' nos tamanhos se quiser usar as quantidades.

In [13]:
# Adicionar octree na nuvem importada com a Pyntcloud
voxelgrid_pyntcloud = pc_pyntcloud.add_structure("voxelgrid", n_x=1, n_y=1, n_z=1, size_x=1.0, size_y=1.0, size_z=1.0)
print(voxelgrid_pyntcloud)

V([369, 369, 369],[1.0, 1.0, 1.0],True)


# Kd-trees & Octrees: Open3D

Kdtrees na Open3D são implementadas utilizando o algoritmo FLANN (Fast Library for Approximate Nearest Neighbors). Diferente da pyntcloud, esse algoritmo não procura todos os pontos mais pŕoximos por força bruta.

In [14]:
# Pinta a nuvem de amarelo:
pc_open3d.paint_uniform_color([0.8, 0.8, 0])

# Cria a kd-tree na nuvem usando flann:
pcd_tree = o3d.geometry.KDTreeFlann(pc_open3d)

# Pinta o ponto de indice 100000 de vermelho:
pc_open3d.colors[100000] = [1, 0, 0]

# Procura os 1000 pontos vizinhos do ponto anterior:
[k, idx, _] = pcd_tree.search_knn_vector_3d(pc_open3d.points[100000],1000)

# Pinta os vizinhos de azul:
np.asarray(pc_open3d.colors)[idx[1:], :] = [0, 0, 1]

# Encontra os vizinhos com distância menor que 5 m:
[k, idx, _] = pcd_tree.search_radius_vector_3d(pc_open3d.points[100000],5.0)

# Pinta os vizinhos dentro dos 5 m de preto:
np.asarray(pc_open3d.colors)[idx[1:], :] = [0, 0, 0]

# Plota a nuvem:
o3d.visualization.draw_geometries([pc_open3d])

A linha baixo cria uma octree com 64 voxels utilizando a Open3D, para isso ela divide um cubo de 1 m³ centrado em [0,0,0] em cubos menores de 0.25 m de largura, o que dá 4x4x4 voxels.

In [15]:
o3d.geometry.VoxelGrid.create_dense([0.0,0.0,0.0],0.25, 1, 1, 1)

geometry::VoxelGrid with 64 voxels.

# Voxialização

Quando temos uma nuvem muito grande é coveniente que se faça uma voxialização para reduzir seu tamanho. Além de deixá-la com densidade uniforme, essa operação consegue reduzir a quantidade de pontos preservando a forma dos objetos. Para o registro a voxialização é crítica pois a operação demanda tempo exponencialmente maior de acordo com o tamanho da nuvem.

Para a voxialização vamos usar a estrutura voxelgrid criada anteriormente com a pyntcloud (voxels com 1m³ de tamanho)


In [16]:
pc_voxializada_pyntcloud = pc_pyntcloud.get_sample(name='voxelgrid_centers',as_PyntCloud=True,voxelgrid_id=voxelgrid_pyntcloud)
print(pc_voxializada_pyntcloud)
# Desenhar nuvem voxializada:
pc_voxializada_pyntcloud.plot()

PyntCloud
83736 points with 0 scalar fields
0 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: 0.8215575814247131, 0.8175594210624695, 0.24738658964633942
Other attributes:



Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(0.8215575814247131, 186.13616293668747, 33.9…

A open3d faz a voxialização direto na nuvem, recebe como parâmetro apenas o tamanho do voxel, quanto maior, menos pontos na nuvem final:

In [17]:
pc_voxializada_open3d = pc_open3d.voxel_down_sample(voxel_size=1.0)
o3d.visualization.draw_geometries([pc_voxializada_open3d])

In [18]:
print(pc_voxializada_open3d)

geometry::PointCloud with 82319 points.


# Filtragem

Filtragens são feitas considerando uma vizinhança mínima que deve existir ao redor de um ponto. Essa vizinhança pode ser definida como um raio geométrico ao redor do ponto ou por um desvio-padrão. No primeiro caso temos o filtro por raio, no segundo o filtro estatístico. Recomenda-se fazer a filtragem sempre após a voxialização.

A filtragem utiliza kd-trees para encontrar vizinhos mais próximos, na pyntcloud precisamos construir a kd-tree da nuvem voxializada que será filtrada:

In [19]:
kdtree_filtragem = pc_voxializada_pyntcloud.add_structure("kdtree")

In [20]:
# RADIOS OUTLIER REMOVAL.
# ENTRADAS: r=raio, k=nº_de_vizinhos, kdtree_id;   
# SAÍDAS: índices dos pontos filtrados.
idx = pc_voxializada_pyntcloud.get_filter(name='ROR',k=6,r=1.5,kdtree_id=kdtree_filtragem)

pts_filtrados = len(idx)-sum(idx)
print("pontos filtrados: %d" %pts_filtrados)

[[0.         1.00221252 1.41577892 1.41577892 1.41739923 2.00440979]
 [0.         1.         1.00219727 1.41576812 1.41576812 1.41577892]
 [0.         1.         1.         1.00219727 1.00221252 1.00229269]
 ...
 [0.         1.         1.         1.00219727 1.41576812 1.41576812]
 [0.         1.         1.00219727 1.41576812 2.         2.00440979]
 [0.         1.00219727 1.41576812 1.41576812 2.00439453 2.23705149]]
pontos filtrados: 3677


In [21]:
# STATISTIICAL OUTLIER REMOVAL.
# ENTRADAS: z_max=desvio_padrao, k=nº_de_vizinhos, kdtree_id;   
# SAÍDAS: índices dos pontos filtrados.
idx = pc_voxializada_pyntcloud.get_filter(name='SOR',k=6,z_max=1.2,kdtree_id=kdtree_filtragem)

pts_filtrados = len(idx)-sum(idx)
print("pontos filtrados: %d" %pts_filtrados)

[ 1.74350599  0.66420449 -0.66907319 ...  0.21815981  1.92244333
  2.62276719]
pontos filtrados: 3101


Os mesmos filtros aplicados na mesma nuvem com os mesmos parâmetros produzem resultados diferentes na Open3D. Isso ocorre porque como mencionado anteriormente, o algoritmo da kd-tree é diferente. Além disso, as voxializações produziram nuvens com quantidades de pontos ligeiramente diferentes, mesmo que o tamanho do voxel utilizado tenha sido o mesmo.

In [22]:
# FILTRO POR RAIO.
# ENTRADAS: Raio e Quantidade mínima de pontos.
# SAÍDAS: nuvem filtrada e índices desses pontos.
pc_filtrada_raio, ind = pc_voxializada_open3d.remove_radius_outlier(nb_points=6, radius=1.8)
print(pc_filtrada_raio)

pts_filtrados = len(pc_voxializada_open3d.points)-len(pc_filtrada_raio.points)
print("Pontos filtrados utilizando um raio de 1.8 m: %d" %pts_filtrados)

geometry::PointCloud with 79704 points.
Pontos filtrados utilizando um raio de 1.8 m: 2615


In [23]:
# FILTRO ESTATÍTISTICO.
# ENTRADAS: Desvio-Padrão e Quantidade mínima de pontos.
# SAÍDAS: nuvem filtrada e índices desses pontos.
pc_filtrada_estat, ind = pc_voxializada_open3d.remove_statistical_outlier(nb_neighbors=6, std_ratio=1.0)
print(pc_filtrada_estat)

pts_filtrados = len(pc_voxializada_open3d.points)-len(pc_filtrada_estat.points)
print("Pontos filtrados utilizando 1 desvio-padrão: %d" %pts_filtrados)

geometry::PointCloud with 79634 points.
Pontos filtrados utilizando 1 desvio-padrão: 2685


Por algum motivo desconhecido a função 'select_down_sample' faz o Kernel do jupyter parar de funcionar, como nenhuma informação retorna, não tem-se como investigar o erro. Portanto, infelizmente não conseguiremos mostrar as duas nuvens (inliers e outliers) com a célula abaixo (rodar ela irá parar o Kernel):

In [ ]:
# Separa as duas nuvens em pontos restantes (inliers) e removidos (outliers)
pontos_restantes = pc_filtrada_estat.select_down_sample(ind)
pontos_removidos = pc_filtrada_estat.select_down_sample(ind, invert=True)

# Coloca cores diferentes nas nuvens:
pontos_restantes.paint_uniform_color([1,0,0])       # Inliers  -> vermelho
pontos_removidos.paint_uniform_color([0.8,0.8,0.8]) # outliers -> cinza

# Desenhar as duas nuvens
o3d.visualization.draw_geometries([pontos_restantes, pontos_removidos])

# Registro de pares de nuvens de pontos

Apenas a Open3D tem funções implementadas para fazer o registro de nuvens de pontos. Essas funções se subdividem em registros globais para alinhamento inicial e registros finos, que consistem em variações do ICP (Iterative Closest Point). O registro é simplesmente o alinhamento de duas nuvens no mesmo referencial, portanto, a sua saída é uma matriz de transformação que inclui uma rotação e uma translação em uma única matriz 4x4.

Para efetuar o registro precisamos de duas nuvens, das normais dos pontos e dos descritores dos pontos. E para o cálculo das normais e dos descritores precisamos da kd-tree. A seguir fazemos todo o pre-processamento:

In [110]:
# Importar duas nuvens com sobreposição e posições translocadas/rotacionadas:
pc_open3d_1 = o3d.io.read_point_cloud("/home/rubens/rubens_virtual_enviroment/nuvens-LiDAR/nuvem_airborne-LiDAR_transladada-1.pcd")
pc_open3d_2 = o3d.io.read_point_cloud("/home/rubens/rubens_virtual_enviroment/nuvens-LiDAR/nuvem_airborne-LiDAR_transladada-2.pcd")

# Voxializar as nuvens:
pc_voxializada_1 = pc_open3d_1.voxel_down_sample(voxel_size=0.1)
pc_voxializada_2 = pc_open3d_2.voxel_down_sample(voxel_size=0.1)

# Filtrar as nuvens (filtro estatístico):
pc_filtrada_1, ind = pc_voxializada_1.remove_statistical_outlier(nb_neighbors=6, std_ratio=1.0)
pc_filtrada_2, ind = pc_voxializada_2.remove_statistical_outlier(nb_neighbors=6, std_ratio=1.0)

# Construir kd-tree para calcular normais (30 cm) e outra para calcular descritores (5 m):
kd_tree_normais     = o3d.geometry.KDTreeSearchParamHybrid(radius=0.2, max_nn=20)
kd_tree_descritores = o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30)

# Calcular normais das nuvens:
normais_1 = pc_filtrada_1.estimate_normals(kd_tree_normais)
normais_2 = pc_filtrada_2.estimate_normals(kd_tree_normais)

# Calcular descritores das nuvens:
pc_fpfh_1 = o3d.registration.compute_fpfh_feature(pc_filtrada_1,kd_tree_descritores)
pc_fpfh_2 = o3d.registration.compute_fpfh_feature(pc_filtrada_2,kd_tree_descritores)

print(pc_filtrada_1)
print(pc_filtrada_2)
print(pc_fpfh_1)
print(pc_fpfh_2)
pc_filtrada_1.paint_uniform_color([0.2, 0.9, 0.2])
pc_filtrada_2.paint_uniform_color([0.2, 0.2, 0.9])
o3d.visualization.draw_geometries([pc_filtrada_1,pc_filtrada_2])

geometry::PointCloud with 10166 points.
geometry::PointCloud with 12376 points.
registration::Feature class with dimension = 33 and num = 10166
Access its data via data member.
registration::Feature class with dimension = 33 and num = 12376
Access its data via data member.


In [111]:
# Função para desenhar o resultado do registro com duas cores diferentes para cada nuvem
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([0.2, 0.9, 0.2])
    target_temp.paint_uniform_color([0.2, 0.2, 0.9])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [112]:
# Limites de convergência do registro = tamanho do voxel utilizado
# O registro é avaliado contando quantos pontos entre as nuvens estão
# abaixo dessa distância  
limite_distancia = 0.1

# Verificar se as nomais dos pontos correspondentes entre a nuvem de origem e destino 
# são semelhantes. Parâmetro em radianos: 0,17 rad = 10 graus, significa que pontos
# homlólogos com normais divergindo acima desse limiar não serão considerados no registro
limiar_normal = o3d.registration.CorrespondenceCheckerBasedOnNormal(0.17)

# Definir o tipo de registro: (ICP ponto-plano)
tipo_registro = o3d.registration.TransformationEstimationPointToPlane()

# Definir parâmetros do algoritmo RANSAC: amostragem mínima de pontos e parada
ransac_n = 4
criterio_parada_RANSAC = o3d.registration.RANSACConvergenceCriteria(4000000, 500)

# Registro Global 1: RANSAC BASED ON FEATURE MATCH

In [113]:
resultado_RANSAC = o3d.registration.registration_ransac_based_on_feature_matching(pc_filtrada_1,
                                                                                  pc_filtrada_2,
                                                                                  pc_fpfh_1,
                                                                                  pc_fpfh_2,
                                                                                  limite_distancia,
                                                                                  tipo_registro,
                                                                                  ransac_n,
                                                                                  [limiar_normal],
                                                                                  criterio_parada_RANSAC)
print(resultado_RANSAC)
# Desenhar nuven 1 registrada na nuvem 2:
draw_registration_result(pc_filtrada_1, pc_filtrada_2,resultado_RANSAC.transformation)
resultado_RANSAC.transformation

registration::RegistrationResult with fitness = 0.000590, inlier_rmse = 0.065582, and correspondence_set size of 6
Access transformation to get result.


array([[ 9.98250294e-01, -3.33876498e-03,  5.90356062e-02,
         0.00000000e+00],
       [-2.16840434e-19,  9.98404586e-01,  5.64648774e-02,
         0.00000000e+00],
       [-5.91299429e-02, -5.63660805e-02,  9.96657672e-01,
         1.12470994e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

# Registro Global 2: FAST-GLOBAL-REGISTRATION (ZHOU, 2016)

In [126]:
# Definir parâmetros do registro FGR
registro_FGR = o3d.registration.FastGlobalRegistrationOption(division_factor = 0.3,        # padrão: 1.4 
                                                             use_absolute_scale = True,    # padrão: False
                                                             decrease_mu = True,           # padrão: False
                                                             maximum_correspondence_distance = 0.15,
                                                             iteration_number = 640,       # padrão: 64
                                                             maximum_tuple_count = 5000,   # padrão: 500
                                                             tuple_scale = 0.9)           # padrão: 0.95)  
# Executar registro FGR
resultado_FGR = o3d.registration.registration_fast_based_on_feature_matching(pc_filtrada_1,
                                                                             pc_filtrada_2, 
                                                                             pc_fpfh_1,
                                                                             pc_fpfh_2,
                                                                             registro_FGR)

# Desenhar nuven=m 1 registrada na nuvem 2
draw_registration_result(pc_filtrada_1, pc_filtrada_2, resultado_FGR.transformation)
print(resultado_FGR)
resultado_FGR.transformation

registration::RegistrationResult with fitness = 0.000000, inlier_rmse = 0.000000, and correspondence_set size of 0
Access transformation to get result.


array([[ -0.16902804,  -0.85284256,   0.49405374,  32.58919827],
       [ -0.80387037,   0.40932615,   0.43156059, -29.17964453],
       [ -0.57028235,  -0.32420932,  -0.75476246,  14.12702063],
       [  0.        ,   0.        ,  -0.        ,   1.        ]])

# Refinamento do registro global 1 por ICP-PONTO-PONTO

In [128]:
limite = 0.1
tipo_registro = o3d.registration.TransformationEstimationPointToPoint()
reg_p2p = o3d.registration.registration_icp(pc_filtrada_1,
                                            pc_filtrada_2,
                                            limite, 
                                            resultado_RANSAC.transformation,
                                            tipo_registro)
draw_registration_result(pc_filtrada_1, pc_filtrada_2, reg_p2p.transformation)
print(reg_p2p)
reg_p2p.transformation

registration::RegistrationResult with fitness = 0.000689, inlier_rmse = 0.061060, and correspondence_set size of 7
Access transformation to get result.


array([[ 0.98324976,  0.0601479 , -0.17205275, -0.75514635],
       [-0.04828534,  0.99621173,  0.07232374,  0.7489775 ],
       [ 0.17575109, -0.06280468,  0.9824292 ,  7.87681687],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

# Refinamento do registro global 1 por ICP-PONTO-PLANO

In [129]:
# ICP PONTO-PLANO
limite = 0.01
tipo_registro = o3d.registration.TransformationEstimationPointToPlane()
reg_p2l = o3d.registration.registration_icp(pc_filtrada_1,
                                            pc_filtrada_2,
                                            limite, 
                                            resultado_RANSAC.transformation,
                                            tipo_registro)
draw_registration_result(pc_filtrada_1, pc_filtrada_2, reg_p2l.transformation)
print(reg_p2l)
reg_p2l.transformation

registration::RegistrationResult with fitness = 0.000000, inlier_rmse = 0.000000, and correspondence_set size of 0
Access transformation to get result.


array([[ 9.98250294e-01, -3.33876498e-03,  5.90356062e-02,
         0.00000000e+00],
       [-2.16840434e-19,  9.98404586e-01,  5.64648774e-02,
         0.00000000e+00],
       [-5.91299429e-02, -5.63660805e-02,  9.96657672e-01,
         1.12470994e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

# Refinamento do registro global 2 por ICP-PONTO-PONTO

In [130]:
limite_distancia = 0.01
tipo_registro = o3d.registration.TransformationEstimationPointToPoint()
resultado_icp = o3d.registration.registration_icp(pc_filtrada_1,
                                                  pc_filtrada_2, 
                                                  limite_distancia,
                                                  resultado_FGR.transformation,
                                                  tipo_registro)
print(resultado_icp)
draw_registration_result(pc_filtrada_1, pc_filtrada_2, resultado_icp.transformation)
resultado_icp.transformation

registration::RegistrationResult with fitness = 0.000000, inlier_rmse = 0.000000, and correspondence_set size of 0
Access transformation to get result.


array([[ -0.16902804,  -0.85284256,   0.49405374,  32.58919827],
       [ -0.80387037,   0.40932615,   0.43156059, -29.17964453],
       [ -0.57028235,  -0.32420932,  -0.75476246,  14.12702063],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

# Refinamento do registro global 2 por ICP-PONTO-PLANO

In [131]:
limite_distancia = 0.01
tipo_registro = o3d.registration.TransformationEstimationPointToPlane()
resultado_icp = o3d.registration.registration_icp(pc_filtrada_1,
                                                  pc_filtrada_2, 
                                                  limite_distancia,
                                                  resultado_FGR.transformation,
                                                  tipo_registro)
print(resultado_icp)
draw_registration_result(pc_filtrada_1, pc_filtrada_2, resultado_icp.transformation)
resultado_icp.transformation

registration::RegistrationResult with fitness = 0.000000, inlier_rmse = 0.000000, and correspondence_set size of 0
Access transformation to get result.


array([[ -0.16902804,  -0.85284256,   0.49405374,  32.58919827],
       [ -0.80387037,   0.40932615,   0.43156059, -29.17964453],
       [ -0.57028235,  -0.32420932,  -0.75476246,  14.12702063],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

# Segmentação

A segmentação de nuvens de pontos consiste em separar estruturas com geometrias iguais. O exemplo mais básico que podemos dar é separar estruturas planas de não planas. Faremos isso calculando os autovalores e com eles calculando índices de planaridade, esfericidade, etc,  para segmentar a nuvem. Utilizaremos a pyntcloud porque nela estes cálculos estão implementados.

In [134]:
# Adicionar estrutura kdtree na nuvem
pc_kdtree  = pc_pyntcloud.add_structure("kdtree")

# Vizinhanças de cada ponto k
k_vizinhos = pc_pyntcloud.get_neighbors(k=20,kdtree=pc_kdtree)

# Adicionar colunas dos autovalores (scalar fields)
autovalores = pc_pyntcloud.add_scalar_field("eigen_values", k_neighbors=k_vizinhos)

# Adicionar Colunas das medidas dimensionais
# Planaridade: (e2-e3/e1)
eigen = pc_pyntcloud.add_scalar_field("planarity",ev=autovalores)
# Anisotropia: (e1-e3/e1)
eigen = pc_pyntcloud.add_scalar_field("anisotropy",ev=autovalores)
# Esfericidade: (e3/e1)
eigen = pc_pyntcloud.add_scalar_field("sphericity",ev=autovalores)
# Variância: (e1*e2*e3)^1/2
eigen = pc_pyntcloud.add_scalar_field("omnivariance",ev=autovalores)
# Entropia: sum[ei*ln(ei)]
eigen = pc_pyntcloud.add_scalar_field("eigenentropy",ev=autovalores)

pc_pyntcloud.points.head()

,x,y,z,e1(21),e2(21),e3(21),planarity(21),anisotropy(21),sphericity(21),omnivariance(21),eigenentropy(21)
0,-53.031326,185.443611,0.658555,0.460452,0.114048,0.000276,0.247088,0.999401,0.000599,0.024376,0.606976
1,-53.250126,185.568611,0.644555,0.416906,0.125244,0.000251,0.299811,0.999398,0.000602,0.023579,0.627024
2,-53.453226,185.631111,0.654555,0.412406,0.116085,0.000261,0.280851,0.999368,0.000632,0.023195,0.617420
3,-53.672026,185.693611,0.650555,0.412406,0.116085,0.000261,0.280851,0.999368,0.000632,0.023195,0.617420
4,-53.859426,185.756111,0.638555,0.412406,0.116085,0.000261,0.280851,0.999368,0.000632,0.023195,0.617420


In [135]:
# Retira do dataFrame todos os pontos que não passam no critério
planos = pc_pyntcloud.points[pc_pyntcloud.points["planarity(21)"] > 0.8]
arvores = pc_pyntcloud.points[pc_pyntcloud.points["planarity(21)"] < 0.2]

In [143]:
# Converte para apenas as primeiras 3 colunas e salva a nuvem de objetos plano
a = planos.values
a = a[:,0:3]
pts = pd.DataFrame(a)
pts.rename(columns={0:'x',1:'y',2:'x'},inplace=True)
pts.to_csv('/home/rubens/rubens_virtual_enviroment/nuvens-LiDAR/nuvem_pts_planos.csv')


In [144]:
# Converte para apenas as primeiras 3 colunas e salva a nuvem de objetos não-planos
a = arvores.values
a = a[:,0:3]
pts = pd.DataFrame(a)
pts.rename(columns={0:'x',1:'y',2:'x'},inplace=True)
pts.to_csv('/home/rubens/rubens_virtual_enviroment/nuvens-LiDAR/nuvem_pts_nao-planos.csv')
# A função de escrita da biblioteca PyntCloud não funciona

In [145]:
# Nuvem dos objetos planos (pontos com planaridade superior que 0.80)
nuvem_planos = pytc.from_file("/home/rubens/rubens_virtual_enviroment/nuvens-LiDAR/nuvem_pts_planos.csv",sep=",",header=0,names=["x","y","z"])
nuvem_planos.plot()

/home/rubens/.local/lib/python3.6/site-packages/pythreejs/traits.py:191: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-4.575900339870704, 179.29775567168153, 19.7…

In [146]:
# Nuvem dos objetos não planos (pontos com planaridade inferior a 0.2)
nuvem_arvores = pytc.from_file("/home/rubens/rubens_virtual_enviroment/nuvens-LiDAR/nuvem_pts_nao-planos.csv",sep=",",header=0,names=["x","y","z"])
nuvem_arvores.plot()

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-5.671152944761193, 181.0905213299242, 32.98…